In [ ]:
%matplotlib inline
import numpy as np
import xarray as xr
from sage3reader import sage3reader

FIG_SIZE = (15, 6)

# Read in the data and index by start_time rather than profile_id

In [ ]:
s3data = sage3reader.multi_path_l2binary_to_dataset('/path/to/data').swap_dims({'event_id': 'start_time'}, inplace=True).sortby('start_time')

# (or open a pre-computed file of the same data)

In [ ]:
s3data = xr.open_dataset('/path/to/data.nc').swap_dims({'event_id': 'start_time'}, inplace=True).sortby('start_time')

# Plot a time-series of the *QA<2* ozone data

In [ ]:
s3data.o3.sel(altitude=slice(10, 55)).where((s3data.o3_qa_flags < 2 )).plot(x='start_time', y='altitude', figsize=FIG_SIZE, robust=True)

# Monthly mean time-series of the ozone over the tropics

In [ ]:
s3data.o3.sel(altitude=slice(10, 55)).where((s3data.o3_qa_flags < 2) & (s3data.start_latitude < 30) & (s3data.start_latitude > -30)).resample('W', dim='start_time').plot(x='start_time', y='altitude', figsize=FIG_SIZE, robust=True)

# Zonal mean, std and count of 2017-09 ozone

In [ ]:
samplemonth = s3data.o3.sel(altitude=slice(10, 55), start_time=slice('20170901', '20171001')).where(s3data.o3_qa_flags < 2)

lat_bins = range(-80, 85, 5)
labels = [lat + 2.5 for lat in lat_bins[:-1]]
binned = samplemonth.groupby_bins('start_latitude', lat_bins, labels=labels).mean(dim='start_time').sortby('start_latitude_bins')
binned.plot(x='start_latitude_bins', y='altitude', robust=True, figsize=FIG_SIZE)

In [ ]:
binned = samplemonth.groupby_bins('start_latitude', lat_bins, labels=labels).std(dim='start_time').sortby('start_latitude_bins')
binned.plot(x='start_latitude_bins', y='altitude', robust=True, figsize=FIG_SIZE)

In [ ]:
binned = samplemonth.groupby_bins('start_latitude', lat_bins, labels=labels).count(dim='start_time').sortby('start_latitude_bins')
binned.plot(x='start_latitude_bins', y='altitude', robust=True, figsize=FIG_SIZE)